In [2]:
%load_ext autoreload
%autoreload 2
from dataset import SpikingDataset
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
#import seaborn as sns
import sys
sys.path.append('../../')
from dataset import Dataset, SpikingDataset, RegSpikingDataset, NewDataset
from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
random.seed(1338)
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh
import pandas as pd

from Loss import ber_loss
from preprocessing import *
import lava.lib.dl.bootstrap as bootstrap

import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation

generating_data = False

%load_ext autoreload

%autoreload 2

torch.__version__
# The coarse network structure is dicated by the Fashion MNIST dataset. 
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    #device = torch.device("cuda")     
else:
    device = torch.device("cpu")
print(device)
tau_mem = 10e-3
tau_syn = 5e-3
time_step = 1e-3
alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

print("init done")
print(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu
init done
cpu


In [3]:
class Network(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Network, self).__init__()

        neuron_params = {
                'threshold'     : 1.25,
                'current_decay' : 1, # this must be 1 to use batchnorm
                'voltage_decay' : 0.03,
                'tau_grad'      : 1,
                'scale_grad'    : 1,
            }
        neuron_params_norm = {
                **neuron_params,
                # 'norm'    : slayer.neuron.norm.MeanOnlyBatchNorm,
            }

        self.blocks = torch.nn.ModuleList([
                bootstrap.block.cuba.Input(neuron_params, weight=1, bias=0), # enable affine transform at input
                bootstrap.block.cuba.Dense(neuron_params_norm, input_size, 32, weight_norm=True, weight_scale=2),
                #bootstrap.block.cuba.Dense(neuron_params_norm, 512, 512, weight_norm=True, weight_scale=2),
                bootstrap.block.cuba.Affine(neuron_params, 32, 2, weight_norm=True, weight_scale=2),
            ])

    def forward(self, x, mode):
        for block, m in zip(self.blocks, mode):
            x = block(x, mode=m)
        return x

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        simulation = h.create_group('simulation')
        simulation['Ts'] = 1
        simulation['tSample'] = self.time_steps
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))

In [4]:
def train_epoch(train_loader, net_snn, optimizer, scheduler, loss_fn):
    all_output = []
    inputs = []
    labels = []
    
    for X, y in train_loader:
        X = X.to(device).float()
        y = y.to(device).float()
        
        re, imag = net_snn(X)
        #re = torch.mean(re, dim=2)
        #imag = torch.mean(imag, dim=2)
        output = torch.cat((re, imag), axis=1)
        
        #output = net_snn(X)
        #output = torch.mean(output, dim=2)
        
        loss = loss_fn(output, y) 
        loss.backward()
        optimizer.step()
        
    scheduler.step()

          

def test(test_loader, net_snn, rc, num_frame = 19):
    all_output = []
    inputs = []
    labels = []
    
    for X, y in test_loader:
        X = X.to(device)
        y = y.to(device)
        
        #output = net_snn(X)
        #output = torch.mean(output, dim=2)
        re, imag = net_snn(X)
        re = torch.mean(re, dim=2)
        imag = torch.mean(imag, dim=2)
        re, imag = torch.sign(y[:, 0]), torch.sign(y[:, 1])
        output = torch.cat((re, imag), axis=1)
    
        print(output[10], y[10])
        all_output.append(output.cpu().detach().numpy())
        labels.append(y.cpu().detach().numpy())
        
    all_output = np.concatenate(all_output, axis=0)
    labels = np.concatenate(labels, axis=0)
    predict_time = rc.time_to_freq(all_output, num_frame, remove_delay=False)
    target_time = rc.time_to_freq(labels, num_frame, remove_delay=False)
    # print(predict_time, target_time)
    return rc.my_new_test(predict_time, target_time)

def train(train_loader, test_loader, net_snn, epochs=100, rc=None, num_frame = 19):
    optimizer = torch.optim.Adam(net_snn.parameters(), lr=5e-3, betas=(0.9,0.999))
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
    #loss_fn = torch.nn.L1Loss()
    loss_fn = torch.nn.MSELoss()
    for i in range(epochs):
        train_epoch(train_loader, net_snn, optimizer, scheduler, loss_fn)
        print(i)
        print(test(test_loader, net_snn, rc, num_frame=19))

In [5]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [10]
nb_inputs = 6

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]

overall_mu_ber = 0.0
overall_teacher_ber = 0.0
overall_base_ber = 0.0
overall_nopath_ber = 0.0
cnt = 0
trained_folder = "trained"
# except 2
packets = [11, 13, 14, 16, 17, 18, 19, 20]  # [11, 13, 14, 16,]
for i in packets:
    print(i)
    rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
            N_reservoir=16,
            spectral_radius=0.2,
            sparsity=0.4,
            noise=1e-6,
            lut_activation=False,  # True,
            tanh_lut=tanh_lut,
            input_scale=25,  #40, #50, # 25,
            reservoir_input_scale = 8,  #4,  #5,
            show_wout=False,
            output_folder= output_folder,
            debug=debug,
            use_fpga= None,
            sock= None,  # usock
            addr = None) # addr

    train_input, train_label, test_input, test_label = rc.run()
    train_input, train_label, test_input, test_label = pre_processing(train_input, train_label)
    train_label = 100 * train_label
    test_label = 100 * test_label
    min_val = np.min(train_input)
    max_val = np.max(train_input)
    print(min_val, max_val)
    train_data = NewDataset(train_input, train_label, sequence_length=nb_inputs, time_step=100, min_val=min_val, max_val=max_val)
    test_data =  NewDataset(test_input, test_label, sequence_length=nb_inputs, time_step=100, min_val=min_val, max_val=max_val)
    train_loader = DataLoader(dataset=train_data, batch_size=256, shuffle=False, drop_last=False)
    test_loader = DataLoader(dataset=test_data, batch_size=256, shuffle=False, drop_last=False)
    net = Network(4, 2).to(device)
    stats = slayer.utils.LearningStats()
    scheduler = bootstrap.routine.Scheduler()
    epochs = 100
    optimizer = torch.optim.Adam(net.parameters(), lr=5e-3, betas=(0.9,0.999))
    scheduler = bootstrap.routine.Scheduler()
    for epoch in range(epochs):
        for i, (input, label) in enumerate(train_loader, 0):
            net.train()
            mode = scheduler.mode(epoch, i, net.training)

            input = input.to(device)
            output = net.forward(input, mode)
            rate = torch.mean(output, dim=-1).reshape((input.shape[0], -1))

            loss = F.l1_loss(rate, label.to(device))

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        
        all_output = []
        labels = []
        for i, (input, label) in enumerate(test_loader, 0):
            net.eval()
            mode = scheduler.mode(epoch, i, net.training)

            with torch.no_grad():
                input = input.to(device)
                output = net.forward(input, mode=scheduler.mode(epoch, i, net.training))
                rate = torch.mean(output, dim=-1).reshape((input.shape[0], -1))
            
                all_output.append(rate.cpu().detach().numpy())
                labels.append(label.cpu().detach().numpy())
                
        all_output = np.concatenate(all_output, axis=0)
        labels = np.concatenate(labels, axis=0)
        print(all_output.shape, labels.shape)
        predict_time = rc.time_to_freq(all_output, 19, remove_delay=False)
        target_time = rc.time_to_freq(labels, 19, remove_delay=False)
    # print(predict_time, target_time)
        print(rc.my_new_test(predict_time, target_time))  

        print(f'\r[Epoch {epoch:2d}/{epochs}] {stats}', end='')

        if mode.base_mode == bootstrap.routine.Mode.SNN:
            scheduler.sync_snn_stat(stats.testing)
            scheduler.update_snn_stat()

        stats.update()
        #stats.save(trained_folder + '/')
    

11
-0.02459430120580924 0.021224323747763597
[Epoch  0/100] Train (1520, 2) (1520, 2)
0.4205043859649123
[Epoch  1/100] Train (1520, 2) (1520, 2)
0.4649122807017544
[Epoch  2/100] Train (1520, 2) (1520, 2)
0.4407894736842105
[Epoch  3/100] Train (1520, 2) (1520, 2)
0.45394736842105265
[Epoch  4/100] Train (1520, 2) (1520, 2)
0.45997807017543857
[Epoch  5/100] Train (1520, 2) (1520, 2)
0.46984649122807015
[Epoch  6/100] Train (1520, 2) (1520, 2)
0.47478070175438597
[Epoch  7/100] Train (1520, 2) (1520, 2)
0.48135964912280704
[Epoch  8/100] Train (1520, 2) (1520, 2)
0.4791666666666667
[Epoch  9/100] Train (1520, 2) (1520, 2)
0.4682017543859649
[Epoch 10/100] Train (1520, 2) (1520, 2)
0.43256578947368424
[Epoch 11/100] Train (1520, 2) (1520, 2)
0.46600877192982454
[Epoch 12/100] Train (1520, 2) (1520, 2)
0.4654605263157895
[Epoch 13/100] Train (1520, 2) (1520, 2)
0.4621710526315789
[Epoch 14/100] Train (1520, 2) (1520, 2)
0.4555921052631579
[Epoch 15/100] Train (1520, 2) (1520, 2)
0.45668

KeyboardInterrupt: 